# E-Commerce analysis using SQL
주요고객/타겟층은 누구이며 수익향상을 위한 그로쓰 전략을 데이터 분석을 통하여 찾고자 한다.
* 참고로, 이 프로젝트에서 활용된 데이터셋에는 2010년 12월 1일부터 2011년 12월 9일까지 약 4000명의 고객에 대한 구매 정보가 포함되어 있습니다.

## 데이터 분석을 위한 환경구축
SQL 쿼리문이 실행될 수 있도록 아래와 같이 환경 구축하고자 한다.

- in-memory SQLite database 만들기
- SQLite database에 'data'라는 table 추가하기
- 쿼리문 실행을 위한 함수를 정의하기
- 쿼리문 작성 및 실행

In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import datetime

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ecommerce-data/data.csv


In [2]:
original_data = pd.read_csv('/kaggle/input/ecommerce-data/data.csv', encoding = 'unicode_escape')
original_data.isnull().sum()  # Description와 CustomerID에 결측치 있음
original_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [3]:
# original_data를 복사한 후 복사본을 대상으로 InvoiceDate 칼럼의 dtype을 object에서 datatime으로 바꿔준다.
data = original_data.copy()
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# in-memory SQLite database 만들기
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo = False)

# SQLite database에 'data'라는 table 추가하기
data.to_sql('data', con = engine)

# 쿼리문 실행을 위한 함수를 정의하기
def run_query(x):
    sql_result = pd.read_sql_query(x, con=engine)
    return sql_result

## 1. 월별 주문 수 통계
시간이 지남에 따라 주문 수가 어떻게 변화하는지 분석하고자 한다.

In [4]:
# 쿼리문 작성 및 실행

# 1. 월별 주문 수 통계
sale_per_month = """
select strftime('%Y %m', InvoiceDate) month,
count(distinct InvoiceNo) num_orders
from data
group by 1
order by 1
"""
run_query(sale_per_month)

,month,num_orders
0,2010 12,2025
1,2011 01,1476
2,2011 02,1393
3,2011 03,1983
4,2011 04,1744
5,2011 05,2162
6,2011 06,2012
7,2011 07,1927
8,2011 08,1737
9,2011 09,2327


위 결과에 따르면, 2011년 12월의 주문 수가 가장 적게 나타났다. 따라서, 12월말까지 통계된 정보인지 확인할 필요가 있다.

In [5]:
last_day_2011_12 = """
select max(InvoiceDate)
from data
where strftime('%Y %m',InvoiceDate) = '2011 12'
"""
run_query(last_day_2011_12)

,max(InvoiceDate)
0,2011-12-09 12:50:00.000000


위와 같이 12월의 마지막 invoice day는 12월 31일이 아닌 12월 9일이다. 따라서 2011년 12월의 기록을 배제할 것이다.

In [6]:
sale_per_month = """
select strftime('%Y %m', InvoiceDate) month, 
count(distinct InvoiceNo) num_orders
from data
where strftime('%Y %m', InvoiceDate) != '2011 12'
group by 1
order by 1
"""
run_query(sale_per_month)

,month,num_orders
0,2010 12,2025
1,2011 01,1476
2,2011 02,1393
3,2011 03,1983
4,2011 04,1744
5,2011 05,2162
6,2011 06,2012
7,2011 07,1927
8,2011 08,1737
9,2011 09,2327


## 2. 월별 총 수익 통계
시간이 지남에 따라 월별 수익이 어떻게 바뀌고 있는지 분석하고자 한다.

In [7]:
# 2. 월별 총 수익, 내림차순으로 정렬
revenue_per_month = """
select strftime('%Y %m', InvoiceDate) month, 
sum(Quantity * UnitPrice) total_revenue
from data
where strftime('%Y %m', InvoiceDate) != '2011 12'
group by 1
order by 2 desc
"""
run_query(revenue_per_month)

,month,total_revenue
0,2011 11,1461756.250
1,2011 10,1070704.670
2,2011 09,1019687.622
3,2010 12,748957.020
4,2011 05,723333.510
5,2011 06,691123.120
6,2011 03,683267.080
7,2011 08,682680.510
8,2011 07,681300.111
9,2011 01,560000.260


2010년 12월 부터 2011년 7월까지는 상승/하락이 반복하다가 2011년 7월 기점으로 이후 상승하는 추세다.

## 3. 월별로 많이 지출하는 고객 추정하기
많이 구매하는 충성고객이 누구인지 추정하고 이를 월별로 랭킹 매겨보고자 한다.

In [8]:
# 3. 매월 가장 많이 지출하는 고객 top3가 누구인지 탐색
client_per_month = """
with a as(select strftime('%Y %m', InvoiceDate) month, 
sum(Quantity * UnitPrice) client_total_revenue,
CustomerID
from data
where strftime('%Y %m', InvoiceDate) != '2011 12' and CustomerID is not null
group by 1, 3
order by 2 desc)

select * 
from (select *, row_number() over(partition by month order by client_total_revenue desc) rank from a) as b
where b.rank <= 3
"""

run_query(client_per_month)

,month,client_total_revenue,CustomerID,rank
0,2010 12,27834.61,18102.0,1
1,2010 12,19950.66,15061.0,2
2,2010 12,13112.52,16029.0,3
3,2011 01,26476.68,14646.0,1
4,2011 01,22998.40,15749.0,2
5,2011 01,18620.20,17450.0,3
6,2011 02,22752.46,14646.0,1
7,2011 02,14022.92,12415.0,2
8,2011 02,10535.48,18102.0,3
9,2011 03,21462.40,14646.0,1


월별로 랭킹을 매겨본 결과, 특정 고객들이 TOP3내에서 빈번하게 출물하는 것을 확인할 수 있다.

## 4. 월별로 수익이 가장 많은 국가 추정하기
매월 수익이 가장 많은 국가는 어느 국가인지 추정하고 이를 월별로 랭킹을 매겨보고자 한다.

In [9]:
# 4. 매월 수익이 가장 많은 국가 top3 탐색
country_per_month = """
with a as(select strftime('%Y %m', InvoiceDate) month, 
         sum(Quantity * UnitPrice) country_total_revenue,
         Country
         from data
         where strftime('%Y %m', InvoiceDate) != '2011 12'
         group by 1, 3
         order by 2 desc)

select * 
from (select *, row_number() over(partition by month order by country_total_revenue desc) rank
     from a) as b
where rank <= 3
"""

run_query(country_per_month)

,month,country_total_revenue,Country,rank
0,2010 12,676742.620,United Kingdom,1
1,2010 12,14562.840,Germany,2
2,2010 12,9575.360,France,3
3,2011 01,434308.300,United Kingdom,1
4,2011 01,26611.160,Netherlands,2
5,2011 01,21656.520,EIRE,3
6,2011 02,408247.910,United Kingdom,1
7,2011 02,22932.110,Netherlands,2
8,2011 02,14627.470,Australia,3
9,2011 03,559707.390,United Kingdom,1


결과에 따르면 영국의 매달 수익이 다른 국가들에 비해서 항상 높다는 사실을 알 수 있다.

## 5. 가장 많이 반품되는 제품 추정하기
가장 많이 반품이 되는 제품이 어떤 것들이 있는지 추정하고자 한다.

In [10]:
# 5. 가장 많이 반품되는 상위 10개 제품
top10_return_products = """
select StockCode, 
Description, 
sum(Quantity) return_quantity,
sum(Quantity * UnitPrice) return_value
from data
where Quantity < 0 and CustomerID is not null
group by StockCode
order by 3
limit 10
"""

run_query(top10_return_products)

,StockCode,Description,return_quantity,return_value
0,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,-168469.60
1,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74494,-77479.64
2,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9367,-298.65
3,M,Manual,-3995,-112165.39
4,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,-3150,-6591.42
5,20971,PINK BLUE FELT CRAFT TRINKET BOX,-2617,-2778.77
6,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-2578,-6624.30
7,21175,GIN + TONIC DIET METAL SIGN,-2030,-3775.33
8,22920,HERB MARKER BASIL,-1527,-841.05
9,22273,FELTCRAFT DOLL MOLLY,-1447,-3512.65


각각 반품되는 제품들에 본질적으로 어떠한 문제가 있어서 반품되었는지 추가적인 분석이 필요하다.

## 6. 주문별 수익금액에 따른 그룹화
주문별 수익금액에 따라 다음과 같이 그룹화하여, 그룹화된 주문유형별 주문수, 주문비중, 주문총수익을 확인하고자 한다.
- 1000이상 => large order
- 100이상~1000미만 => normal order
- 0이상~100미만 => small order
- 0일 경우 => free order
- 0미만 => return order

In [11]:
order_rev = """
with a as(select InvoiceNo,
sum(Quantity * UnitPrice) total_revenue
from data
group by 1
order by 2 desc),
b as(select case when total_revenue >= 1000 then 'Large Order'
                 when total_revenue >= 100 and total_revenue < 1000 then 'Normal Order'
                 when total_revenue > 0 and total_revenue < 100 then 'Small Order'
                 when total_revenue = 0 then 'Free Order'
                 else'Return Order'
                 end as Order_Group,
            InvoiceNo, 
            total_revenue
            from a),
c as (select Order_Group, 
             count(InvoiceNo) Num_Order, 
             (100 * count(InvoiceNo) / (select count(distinct InvoiceNo) from data)) percent_of_total_orders,
             sum(total_revenue) Total_Revenue
      from b
      group by 1)
      
select * from c order by Num_Order desc
"""

run_query(order_rev)

,Order_Group,Num_Order,percent_of_total_orders,Total_Revenue
0,Normal Order,15397,59,5426436.692
1,Return Order,3838,14,-918936.610
2,Small Order,2721,10,116642.781
3,Free Order,2102,8,0.000
4,Large Order,1842,7,5123605.071


- 위 결과에 따르면, 주문의 59%는 100이상~1000미만인 Normal Order임을 알 수 있다.
- Large Order는 총 주문의 7%밖에 안 되지만, Normal Order의 수익과 거의 비슷하게 나타났다.
- 반품비율은 14% 나타남을 알 수가 있다.

결론적으로 반품비율을 줄이기 위한 전략이 필요하며 Large Order 주문비율이 많지는 않지만 수익성이 크기 때문에 수익향상을 위해서 적극적으로 그로쓰할 필요는 있다.